In [1]:
from ultralytics import YOLO
from PIL import Image
import plotly.express as px
import plotly.graph_objects as go
import imageio.v3 as iio
import numpy as np
from matplotlib import pyplot as plt
from plotly.subplots import make_subplots
import helper_functions as hf
import pandas as pd
from pathlib import Path
import pathlib
import platform
plt2 = platform.system()
if plt2 == 'Windows': pathlib.PosixPath = pathlib.WindowsPath

In [2]:
import torchvision
import torch

print("Cuda:",torch.cuda.is_available())
torchvision.__version__

Cuda: True


'0.14.0+cu117'

Run in cmd
### yolo segment train data=yolov8-seg-test.yaml model=yolov8n-seg.pt epochs=10 imgsz=1024

In [3]:

# Load a model
# model = YOLO('yolov8x-seg.yaml')  # build a new model from YAML

# this one
#model = YOLO('yolov8x-seg.pt')  # load a pretrained model (recommended for training)

# model = YOLO('yolov8n-seg.yaml').load('yolov8n.pt')  # build from YAML and transfer weights
# model = YOLO('YOLOv8x-seg.yaml').load('YOLOv8x-seg.pt')  # build from YAML and transfer weights


# This one for training
# results = model.train(data='yolov8-seg-test.yaml', epochs=10, imgsz=1024)

In [14]:

# Load a model
# model = YOLO('yolov8n-seg.pt')  # load an official model
# model = YOLO('runs\\segment\\train\\weights\\best.pt')  # load a custom model
model = YOLO('runs\\segment\\train13\\weights\\last.pt')  # load a custom model

# Predict with the model
results = model("D:\Master\MasterProject\model_creation\images\\2020-02-05_15.41.32-NAs-T1354-GFP_Burst_Frame_6.png")  # predict on an image
# results = model('D:/Master/MasterProject/dataset_creation/datasets/full_frame_dataset/train/2020-02-05_15.41.32-NAs-T1354-GFP_Burst_Frame_6.png')  # predict on an image


image 1/1 D:\Master\MasterProject\model_creation\images\2020-02-05_15.41.32-NAs-T1354-GFP_Burst_Frame_6.png: 1024x1024 (no detections), 11.0ms
Speed: 2.0ms preprocess, 11.0ms inference, 0.0ms postprocess per image at shape (1, 3, 1024, 1024)


In [24]:
d: np.array = results[0].plot(conf=False, line_width=0, font_size=0,img=np.zeros((1024, 1024, 3), dtype=np.uint8), kpt_radius=0, kpt_line=False, labels=False, boxes=False, masks=True, probs=False)

np.max(d)

0

# Model predicted bounding boxes & masks

In [7]:
# Show the results
for r in results:
    im_array = r.plot()  # plot a BGR numpy array of predictions
    
    boxes = r.boxes  # Boxes object for bbox outputs
    masks = r.masks  # Masks object for segmentation masks outputs
    probs = r.probs  # Class probabilities for classification outputs

    im = iio.imread('D:/Master/MasterProject/dataset_creation/datasets/full_frame_dataset/train/2020-02-05_15.41.32-NAs-T1354-GFP_Burst_Frame_6.png')
    fig = px.imshow(im)

    print(r.masks)
    break
    # for mask, box in zip(r.masks.xy, r.boxes):
    #     points = np.int32(mask)
        
    #     # BB
    #     bb = box.xyxy[0]
    #     fig.add_shape(type='rect', 
    #                   x1=bb[0],y1=bb[1], x0=bb[2], y0=bb[3],
    #                   line=dict(color='red', width=2),
    #                   fillcolor='rgba(255, 0, 0, 0.2)',
    #                   )
    #     # Mask
    #     fig.add_trace(go.Scatter(x=mask[:,0], y=mask[:,1], marker=dict(color='cyan', size=4), fill='toself'))

    # fig.show()

None


# Model outputs

In [26]:
analyzed_granule_results = pd.read_hdf(Path("D:/Master/MasterProject/model_creation/datasets/analyzed_granule_data/2020-02-05_15.41.32-NAs-T1354-GFP_Burst.h5"), mode="r", key="fourier")
valid_granule_fourier = analyzed_granule_results[(analyzed_granule_results['valid'] == True) & (analyzed_granule_results['frame'] == 6)]['granule_id']
valid_granule_ids = valid_granule_fourier.unique()

fig = make_subplots(
    rows=1, cols=3, horizontal_spacing=0.01, vertical_spacing=0,subplot_titles=('Base image + True labels','Full prediction', 'Predicted Masks'))
    # specs=[[{"type": "scatter"}, {"type": "bar"}],
        #    [{"type": "image"}, {"type": "surface"}]])
im = iio.imread('D:/Master/MasterProject/dataset_creation/datasets/full_frame_dataset/train/2020-02-05_15.41.32-NAs-T1354-GFP_Burst_Frame_6.png')
base_image = px.imshow(im)

result_masks = results[0].plot(conf=False, line_width=0, font_size=0,img=np.zeros((1024, 1024, 3), dtype=np.uint8), kpt_radius=0, kpt_line=False, labels=False, boxes=False, masks=True, probs=False)
result_full = results[0].plot()
image_result_masks = px.imshow(result_masks)
image_result_full = px.imshow(result_full)
fig.add_trace(base_image.data[0], 1, 1)
fig.add_trace(image_result_masks.data[0], 1, 3)
fig.add_trace(image_result_full.data[0], 1, 2)


# Add Valid granule borders
for id in valid_granule_ids:
    granule_fourier2 = analyzed_granule_results[(analyzed_granule_results['granule_id'] == id) & (analyzed_granule_results['frame'] == 6)]
    
    xs, ys = hf.get_coords(granule_fourier2)
    # Pixel coords
    ys_coords = np.round(ys, 0)
    xs_coords = np.round(xs, 0)

    fig.add_trace(go.Scatter(y=ys[::4], x=xs[::4], marker=dict(color='red', size=16), name=f"True Border - Granule {id}"), col=1, row=1)
    fig.add_trace(go.Scatter(y=ys_coords, x=xs_coords, marker=dict(color='cyan', size=8), fill='toself', name=f"Pixel Border - Granule {id}"), col=1, row=1)

fig.update_layout(title_text="Model output", title_x=0.5,width=1600, height=800, showlegend=False, font_size=11)
# fig.add_image(new)
# mask_img = result.plot(conf=False, line_width=0, font_size=0, img=np.zeros((img_height, img_width, 3), dtype=np.uint8), kpt_radius=0, kpt_line=False, labels=False, boxes=False, masks=True, probs=False)

# Comments/thoughts



### Dataset
Training data only consists of granules we already know how to identify. -> Need to find a way to teach the model to recognize new granules.

    - Modify Granule Explorer to give estimates for non-circular granule's?
    - Create synthetic data? Simulate granule shapes and create custom data samples?

    - Note: This test model is only trained on one experiment. Since all frames are fairly similar, model accuracy are most likely lower. 

### Model's and model size
Currently only able to use smallest YOLOv8 model. (The 'nano' version)

    - If more computing power is available -> Train larger model. Might be able to pick up on more complex underlying data structures for better precision.

    - Test more model architectures. YOLOv8 works, can also do 
        - Segment Anything Model
        - Detectron2 Mask-RCNN
        - YOLO-NAS

### Switch strategy? (Problem with output accuracy, what do we want to achieve?)
The full microscope images are large with many small 'features'. Granules are only 20-40 pixel wide/high, which is difficult for networks to work with.
Instead, crop out all potential granules and classify them only? Skipping all the background.

    - Can then also upscale each granule by 10x, giving us more accuracy.






Sper ocmputer uib/uio -> nrec